# Homework 2 - TF-IDF Classifier

Ваша цель обучить классификатор который будет находить "токсичные" комментарии и опубликовать решения на Kaggle [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

В процессе обучения нужно ответить на ***[вопросы](https://docs.google.com/forms/d/e/1FAIpQLSd9mQx8EFpSH6FhCy1M_FmISzy3lhgyyqV3TN0pmtop7slmTA/viewform?usp=sf_link)***

Данные можно скачать тут - https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data



In [1]:
import numpy as np
import pandas as pd
import nltk

from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')

Стадартными подходами для анализа текста являются [Bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) и его модификация [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Они реалзованны в `sklearn` в виде [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

Более подробней про них можно посмотреть [тут](https://github.com/udsclub/workshop/blob/master/notebooks/UDS-workshop-feature-extraction-and-engineering.ipynb)

In [3]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [18]:
# Попробуйте разные Vectorizer и разные размеры n-gramm, стоп-слова, обрезку редких слов, обрезку слишком частых слов
word_vectorizer = CountVectorizer(analyzer='word', strip_accents='unicode', token_pattern=r'\w{1,}') # TfidfVectorizer или CountVectorizer

In [5]:
char_vectorizer = CountVectorizer(analyzer='char', strip_accents='unicode', ngram_range=(1, 5))

In [23]:
%%time
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

CPU times: user 41.4 s, sys: 2.11 s, total: 43.5 s
Wall time: 46.5 s


In [7]:
%%time
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

CPU times: user 12min 57s, sys: 54 s, total: 13min 52s
Wall time: 14min 27s


In [24]:
tfidf_trans = TfidfTransformer(sublinear_tf=True)

In [25]:
tfidf_trans.fit(train_word_features)
train_word_features = tfidf_trans.transform(train_word_features)

In [26]:
tfidf_trans.fit(test_word_features)
test_word_features = tfidf_trans.transform(test_word_features)

In [11]:
tfidf_trans.fit(train_char_features)
train_char_features = tfidf_trans.transform(train_char_features)

In [12]:
tfidf_trans.fit(test_char_features)
test_char_features = tfidf_trans.transform(test_char_features)

In [27]:
train_features = hstack([train_char_features, train_word_features])

In [28]:
test_features = hstack([test_char_features, test_word_features])

# Task 1 - the most frequent term

In [23]:
word_vectorizer = CountVectorizer()

In [24]:
all_mx = word_vectorizer.fit_transform(all_text)

In [25]:
frequency = list(zip(word_vectorizer.get_feature_names(), np.asarray(all_mx.sum(axis=0)).ravel()))

In [26]:
frequency = sorted(frequency, key=lambda x: -x[1])

In [27]:
frequency[0]

('the', 918456)

Для классификации будем использовать логистическую регрессию [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [29]:
log_clf = LogisticRegression(C=2, class_weight='balanced') # Попробуйте разные параметры, найтдите оттимальные на кросс-валидации

In [91]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [92]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words="english")),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(C=2))
])

In [10]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (5, 10)
}

In [12]:
gs_clf = GridSearchCV(text_clf, parameters, scoring='roc_auc')

In [30]:
train_pred = train['toxic']

In [31]:
log_clf.fit(train_features, train_pred)

cv_score = np.mean(cross_val_score(log_clf, train_features, train_pred, scoring='roc_auc'))

print('CV score for class toxic is {:.4f}'.format(cv_score))

CV score for class toxic is 0.9796


Будем тренировать по одному классификатору на каждый класс. 

Что бы провалидировать качество модели воспользуемся функцией [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

In [34]:
def get_score(clf):
    scores= []

    for class_name in class_names:
        train_target = train[class_name]

        clf.fit(train_features, train_target)

        cv_score = np.mean(cross_val_score(clf, train_features, train_target, cv=3, scoring='roc_auc'))

        print('CV score for class {} is {:.4f}'.format(class_name, cv_score))
        scores.append(cv_score)

    print('Total score is {:.4f}'.format(np.mean(scores)))

In [ ]:
get_score(log_clf)

CV score for class toxic is 0.9796
CV score for class severe_toxic is 0.9888
CV score for class obscene is 0.9915


Попробуйте подобрать лучшие параметры для `word_vectorizer` и `classifier` оптимизируя метрику [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)


---

Опубликуйте лучшие решение на [Kaggle Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/submit)

In [ ]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [ ]:
for class_name in class_names:
    train_target = train[class_name]
    log_clf.fit(train_features, train_target)
    
    submission[class_name] = log_clf.predict_proba(test_features)[:, 1]    

In [ ]:
submission.to_csv('submission.csv', index=False)